In [ ]:
import gdal
import os
import random

#读取tif数据集
def readTif(fileName):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName+"文件无法打开")
    return dataset


Landset_Path = r"C:\Users\admin\Desktop\orginal tif\2020-07-27-00_00_2020-07-27-23_59_Sentinel-2_L1C_True_color.tif"
LabelPath = r"C:\Users\admin\Desktop\label\2020-bjql1.tif"
txt_Path = r"D:\data.txt"

##########################################################  读取图像数据
dataset = readTif(Landset_Path)
Tif_width = dataset.RasterXSize #栅格矩阵的列数
Tif_height = dataset.RasterYSize #栅格矩阵的行数
Tif_bands = dataset.RasterCount #波段数
Tif_geotrans = dataset.GetGeoTransform()#获取仿射矩阵信息
Landset_data = dataset.ReadAsArray(0,0,Tif_width,Tif_height)
dataset = readTif(LabelPath)
Label_data = dataset.ReadAsArray(0,0,Tif_width,Tif_height)


# 写之前，先检验文件是否存在，存在就删掉
if os.path.exists(txt_Path):
    os.remove(txt_Path)
# 以写的方式打开文件，如果文件不存在，就会自动创建
file_write_obj = open(txt_Path, 'w')

####################################################首先收集植被类别样本，
####################################################遍历所有像素值，
####################################################为植被的像元全部收集。
count = 0
for i in range(Label_data.shape[0]):
    for j in range(Label_data.shape[1]):
        #  我设置的植被类别在标签图中像元值为1
        if(Label_data[i][j] == 1):
            var = ""
            for k in range(Landset_data.shape[0]):
                var = var + str(Landset_data[k][i][j])+","
            var = var + "Vegetation"
            file_write_obj.writelines(var)
            file_write_obj.write('\n')
            count = count + 1
   
####################################################其次收集非植被类别样本，
####################################################因为非植被样本比植被样本多很多，
####################################################所以采用在所有非植被类别中随机选择非植被样本，
####################################################数量与植被样本数量保持一致。
Threshold = count
count = 0
# for i in range(10000000000):
#     X_random = random.randint(0,Label_data.shape[0]-1)
#     Y_random = random.randint(0,Label_data.shape[1]-1)
#     #  我设置的非植被类别在标签图中像元值为0
#     if(Label_data[X_random][Y_random] == 0):
for i in range(Label_data.shape[0]):
    for j in range(Label_data.shape[1]):
        #  我设置的非植被类别在标签图中像元值为23
        if(Label_data[i][j] == 23):
            var = ""
            for k in range(Landset_data.shape[0]):
                var = var + str(Landset_data[k][i][j])+","
            var = var + "Non-Vegetation"
            file_write_obj.writelines(var)
            file_write_obj.write('\n')
            count = count + 1
        if(count == Threshold):
            break

file_write_obj.close()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn import model_selection
import pickle 

#  定义字典，便于来解析样本数据集txt
def Iris_label(s):
    it={b'Vegetation':0, b'Non-Vegetation':1}
    return it[s]

path=r"E:\all.txt"
SavePath = r"D:\model.pickle"


#  1.读取数据集
print("开始加载数据....")
data=np.loadtxt(path, dtype=float, delimiter=',', converters={3:Iris_label} )   # 改为4
print("数据加载成功.....   ",data.shape)
#  converters={7:Iris_label}中“7”指的是第8列：将第8列的str转化为label(number)

#  2.划分数据与标签
x,y=np.split(data,indices_or_sections=(3,),axis=1) #x为数据，y为标签
x=x[:,0:3] #选取前7个波段作为特征
train_data,test_data,train_label,test_label = model_selection.train_test_split(x,y, random_state=1, train_size=0.9,test_size=0.1)
print("训练数据与验证数据划分成功....." )

#  3.用100个树来创建随机森林模型，训练随机森林
print("开始训练....." )
classifier = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
classifier.fit(train_data, train_label.ravel())#ravel函数拉伸到一维


#  4.计算随机森林的准确率
print("训练集：",classifier.score(train_data,train_label))
print("测试集：",classifier.score(test_data,test_label))

#  5.保存模型
#以二进制的方式打开文件：
file = open(SavePath, "wb")
#将模型写入文件：
pickle.dump(classifier, file)
#最后关闭文件：
file.close()